In [1]:
import pandas as pd
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def FindAlternateGroups(url_template):
    
    page = 1
    while True:

        # Create a session and set headers
        session = requests.Session()
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            # Add other necessary headers here
        }
        session.headers.update(headers)

        url = url_template.format(page)

        response = requests.get(url)
        product_data = response.json()

        # If the product data is empty, it means we have reached the last page
        if not product_data["products"]:
            break

        page += 1

    total_pages = page - 1
    
 ############################################################################################################   
    title = []


    for j in range(total_pages):

        # Create a session and set headers
        session = requests.Session()
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            # Add other necessary headers here
        }

        session.headers.update(headers)

        url = url_template.format(j)
        # Send a GET request to the URL
        response = requests.get(url)

        # Extract the JSON data from the response
        product_data = response.json()

        for i in range(len(product_data["products"])):
            product = product_data["products"][i]
            title.append(product["title"])
            
 #################################################################################################################

    # Find the index of "/collections"
    index = url_template.index("/collections")

    # Extract the substring before "/collections"
    substring = url_template[:index]

    link = []

    for j in range(total_pages):

        # Create a session and set headers
        session = requests.Session()
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            # Add other necessary headers here
        }

        session.headers.update(headers)

        url = url_template.format(j)
        # Send a GET request to the URL
        response = requests.get(url)

        # Extract the JSON data from the response
        product_data = response.json()

        for i in range(len(product_data["products"])):
            product = product_data["products"][i]
            sub_url = substring+'/products/{}'
            link.append(sub_url.format(product["handle"]))
            
            
 ########################################################################################################################


    title2 = []

    for text in title:

        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(text.lower())
        tokens = ' '.join([lemmatizer.lemmatize(token) for token in tokens if token.isalnum()])
        title2.append(tokens)
        
    count_vectorizer = CountVectorizer()
    vector_matrix = count_vectorizer.fit_transform(title2)

    cosine_similarity_matrix = pd.DataFrame(cosine_similarity(vector_matrix))
    
    def group_similar_elements(similarity_matrix):
        N = len(similarity_matrix)
        groups = []

        visited = set()

        for i in range(N):
            if i not in visited:
                group = [i]
                visited.add(i)

                for j in range(i+1, N):
                    if similarity_matrix[i][j] > 0.5:   # threshold of 0.5 is selected for similarity. 
                        group.append(j)                 # for more stricture on can increase the threshold parameter.
                        visited.add(j)

                if len(group) > 1:
                    groups.append(group)

        return groups


    similar_index = group_similar_elements(cosine_similarity_matrix)
    
    similar_titles = []
    for j in range(len(similar_index)):
        s = []
        for i in similar_index[j]:
            s.append(title[i])
        similar_titles.append(s)
        
        
    similar_titles_link = []
    for j in range(len(similar_index)):
        s = []
        for i in similar_index[j]:
            s.append(link[i])
        similar_titles_link.append(s)
        
    json_data = []
    
    for group,l in zip(similar_titles_link,range(len(similar_titles_link))):
        json_group = {f"product alternates{l}": []}
        for element_index in group:
            json_group[f"product alternates{l}"].append(element_index)
        json_data.append(json_group)
        
    
    return json_data

In [3]:
FindAlternateGroups('https://www.boysnextdoor-apparel.co/collections/all/products.json?page={}')

[{'product alternates0': ['https://www.boysnextdoor-apparel.co/products/a-p-c-daniela-logo-print-denim-tote-bag',
   'https://www.boysnextdoor-apparel.co/products/a-p-c-daniela-logo-print-denim-tote-bag',
   'https://www.boysnextdoor-apparel.co/products/the-north-face-denim-tote-bag-black',
   'https://www.boysnextdoor-apparel.co/products/the-north-face-denim-tote-bag-indigo']},
 {'product alternates1': ['https://www.boysnextdoor-apparel.co/products/and-wander-dyneema-stuffsack',
   'https://www.boysnextdoor-apparel.co/products/and-wander-dyneema-stuffsack']},
 {'product alternates2': ['https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-charcoal',
   'https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-navy',
   'https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-charcoal',
   'https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-navy']},
 {'product alternates3': ['https://www.boysnextdoor-apparel.